In [ ]:
####################
#set up packages 
####################
import ee                     
ee.Initialize()
import pandas as pd
import csv
import time
from datetime import date
import numpy as np
import os, sys

In [ ]:
####################
#open track files
####################
files = os.listdir('C:/Users/Y/Desktop/CHES/hurricane/data/ibtracs/byCategory/h1_2001-2017/')

In [ ]:
start_time_total = time.time() #start runtime record

####################
#User defined functtions
####################
##create not overlapping zones along the hurricane track
def createBuffer(dist1,dist2,vector):
    buffer = vector.buffer(dist1).difference(vector.buffer(dist2),ee.ErrorMargin(1))
    return buffer  

##get patch vectors
def getImgVec(image,roi,threshold,resolution):
    vectors = image.clip(roi).gt(threshold)\
              .updateMask(image.clip(roi).gt(threshold))\
              .reduceToVectors(geometry= roi.geometry(),
                               scale= resolution,
                               geometryType= 'Polygon',
                               eightConnected=True,  #True or False?
                               maxPixels=1e12)
    return vectors

####################
#loop hurricane track
####################
for file in files:
    start_time_hurricane = time.time() 
    data = pd.read_csv('C:/Users/Y/Desktop/CHES/hurricane/data/ibtracs/byCategory/h1_2001-2017/'+ file) 
    Hyear = int(data.iloc[0]['Year'])
    Hname = data.iloc[0]['Name']
    data = data[['Longitude','Latitude']]
    coors = data.values.tolist()
    track = ee.Feature(ee.Geometry.LineString(coors),{'Name':Hname, 'Year': Hyear})
    print(track.getInfo()['properties'])

    trackBuffer = track.buffer(200000)    #create zones along the track
    buffer10k = track.buffer(10000)
    buffer20k = createBuffer(20000,10000,track)
    buffer30k = createBuffer(30000,20000,track)
    buffer40k = createBuffer(40000,30000,track)
    buffer50k = createBuffer(50000,40000,track)
    buffer60k = createBuffer(60000,50000,track)
    buffer70k = createBuffer(70000,60000,track)
    buffer80k = createBuffer(80000,70000,track)
    buffer90k = createBuffer(90000,80000,track)
    buffer100k = createBuffer(100000,90000,track)
    
####################
#load the vegetation map
####################
    mangrove_raw = ee.ImageCollection("LANDSAT/MANGROVE_FORESTS")\
                     .first()                            #2000
    vege_rsl = 250
    mangrove_raw_vec = mangrove_raw.clip(trackBuffer.geometry()).eq(1)\
                                   .updateMask(mangrove_raw.clip(trackBuffer.geometry()).eq(1))\
                                   .reduceToVectors(geometry= trackBuffer.geometry(),
                                                    scale= vege_rsl,
                                                    geometryType= 'Polygon',
                                                    eightConnected=True,  
                                                    maxPixels=1e13)               #convert image to vector
    
    if mangrove_raw_vec.size().getInfo() == 0:   #skip if not intersect with mangrove areas
        continue
    
    mangrove_raw_vec_lg = mangrove_raw_vec.filterMetadata('count','greater_than',16) #exclude patch smaller than 1km2, 10^6m2
    print('Vegetation pathches after filtering: ' + str(mangrove_raw_vec_lg.size().getInfo()))
    print('Total pixels after filtering: ' + str(mangrove_raw_vec_lg.aggregate_sum('count').getInfo()))
    
    if mangrove_raw_vec_lg.size().getInfo() == 0:   #skip if no mangrove areas left
        continue
        
    if mangrove_raw_vec_lg.aggregate_max('count').getInfo() < 160:
        continue           #skip is the largest patch is smaller than 10km2
    
    mangrove = mangrove_raw_vec_lg.reduceToImage(['count'],'mean') #creat a map of vegetation with pathes size as pixel values
    mgvec10k = getImgVec(mangrove,buffer10k,1e6/vege_rsl/vege_rsl,vege_rsl) #vegetation zones
    mgvec20k = getImgVec(mangrove,buffer20k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec30k = getImgVec(mangrove,buffer30k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec40k = getImgVec(mangrove,buffer40k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec50k = getImgVec(mangrove,buffer50k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec60k = getImgVec(mangrove,buffer60k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec70k = getImgVec(mangrove,buffer70k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec80k = getImgVec(mangrove,buffer80k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec90k = getImgVec(mangrove,buffer90k,1e6/vege_rsl/vege_rsl,vege_rsl)
    mgvec100k = getImgVec(mangrove,buffer100k,1e6/vege_rsl/vege_rsl,vege_rsl)

    img_rsl = 250

    imgCltList = ["MODIS/006/MOD13Q1","MODIS/006/MYD13Q1"]
    vectorList = [mgvec10k, mgvec20k, mgvec30k, mgvec40k, mgvec50k, mgvec60k, mgvec70k, mgvec80k, mgvec90k, mgvec100k]
    treshold = 3000 #set up treshold

    for index in (['EVI', 'numberPatches','totalArea','maxPatch','meanPatch']): 
        print(index)
        file = 'C:/Users/Y/Desktop/CHES/hurricane/result/downloaded/' + Hname + str(Hyear) + '_' + index + '_'\
                   + date.today().strftime('%Y-%m-%d') + '.csv'  #set up output file
        with open(file, 'w',newline='') as output:
            writer = csv.writer(output, delimiter=',')
            writer.writerow(['Collection'] + ['Index'] + ['Variable'] + ['Value'])   #write row names

        nameList =  [index + 'mg' + str(s) + 'k' for s in np.arange(10,210,10)] #create the df
        start_time = time.time()
                
        for v in range(0,len(vectorList)):
            vector = vectorList[v]
            name = nameList[v]
            print((v+1)*10, 'km')
                    
            for ic in range(0,len(imgCltList)):
        
                for year in range(Hyear-2,Hyear+4):
                    df = pd.DataFrame(columns=(['Collection'] + ['Index'] +['Variable'] + ['value'])) #create df 
                    imgClt = ee.ImageCollection(imgCltList[ic])\
                               .filterDate(str(year)+'-01-01', str(year)+'-12-31')\
                               .select(['EVI']) 
                    print(imgCltList[ic], 'in', year,'with', imgClt.size().getInfo() ,'images') 
                    if imgClt.size().getInfo() == 0:
                        continue
                    if index=='EVI':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index :x.clip(trackBuffer)\
                                                .reduceRegion(geometry=vector,reducer=ee.Reducer.mean(),scale=img_rsl)\
                                                .get('EVI')}))
                
                    if index=='numberPatches':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
                                                ,vector,treshold,img_rsl).size()}))     
                    
                    if index=='totalArea':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
                                                ,vector,treshold,img_rsl).aggregate_sum('count')}))     
                    
                    if index=='maxPatch':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
                                                ,vector,treshold,img_rsl).aggregate_max('count')}))        
                   
                    if index=='meanPatch':
                        imgClt_addedProperty = imgClt.map(lambda x: 
                                 x.set({index : getImgVec(x.clip(trackBuffer)\
                                                ,vector,treshold,img_rsl).aggregate_mean('count')}))     
                
                    listOfImages = imgClt_addedProperty.toList(imgClt_addedProperty.size()).getInfo()
                    indices = [x['properties']['system:index'] for x in listOfImages]
                    df['Index'] = indices
                    df['Collection'] = imgCltList[ic]
                    df['Variable'] = name
                    values = [-9999] * len(listOfImages)

                    for v in range(0,len(listOfImages)):
                        if index in listOfImages[v]['properties']: 
                            values[v] = listOfImages[v]['properties'][index]
                    df['value'] = values
          
                    with open(file, 'a',newline='') as output:
                        writer = csv.writer(output, delimiter=',')       #output data
                        for row in range(0,len(df.index)):
                            writer.writerow(df.loc[row,])
                  
        end_time = time.time()
        print('Runtime for', index, ':',(end_time-start_time)/60,'min')  
    end_time_hurricane = time.time()
    print('Runtime for',Hname,':',(end_time_hurricane-start_time_hurricane)/60,'min')    
end_time_total = time.time()
print('Total runtime:',(end_time_total-start_time_total)/60,'min') 